In [31]:
"""# 교차 검증과 그리드 서치"""

"""## 검증 세트"""
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [32]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [33]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [34]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [35]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [36]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [37]:
"""## 교차 검증"""
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.00699234, 0.00700116, 0.00561833, 0.00500035, 0.00500011]), 'score_time': array([0.        , 0.00099826, 0.        , 0.        , 0.        ]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [38]:
import numpy as np
print(np.mean(scores['test_score']))

0.855300214703487


In [39]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [40]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [41]:
"""## 하이퍼파라미터 튜닝"""
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [42]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params,n_jobs=-1)

In [43]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [44]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


In [45]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [46]:
print(gs.cv_results_['mean_test_score'])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [47]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [48]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001), "max_depth" : range(5, 20, 1), 'min_samples_split' : range(2, 100, 10)}

In [49]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [50]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [51]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


In [52]:
"""### 랜덤 서치"""
from scipy.stats import uniform, randint
rgen = randint(0, 10)
rgen.rvs(10)

array([2, 8, 8, 0, 5, 3, 6, 9, 5, 4])

In [53]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 85, 119,  96, 102, 108, 125,  91,  82,  84, 108], dtype=int64))

In [54]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.05809304, 0.45137287, 0.45934401, 0.29016422, 0.09580896,
       0.30087423, 0.92010192, 0.21755227, 0.39158506, 0.07935544])

In [55]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001), 'max_depth': randint(20, 50), 'min_samples_split': randint(2, 25), 'min_samples_leaf': randint(1, 25),}

In [56]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001C36193D610>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001C361BF8DF0>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001C361985220>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x000001C361BF8B20>},
                   random_state=42)

In [57]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [58]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [59]:
dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.86
